In [1]:
import os
import glob
import numpy as np
from shapely.geometry import Polygon

img_dir = "./DOTAv10/data_train_2_5_gb/split_ss_dota_1024_200/val/images"
ann_dir = "./DOTAv10/data_train_2_5_gb/split_ss_dota_1024_200/val/annfiles"
pred_dir = "./DOTAv10/data_train_2_5_gb/split_ss_dota_1024_200/val/pseudo_obb_labelTxt_dota_pointobb"

img_list = sorted(os.listdir(img_dir))
ann_list = sorted(os.listdir(ann_dir))
pred_ann_list = sorted(os.listdir(pred_dir))

print(len(img_list))
print(len(ann_list))
print(len(pred_ann_list))

501
501
501


### using many to one matching strategy

In [2]:
def parse_dota_annotation(file_path):
    """Parse a DOTA annotation file and return a list of polygons with categories."""
    objects = []
    with open(file_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) < 9:
                continue
            coords = list(map(float, parts[:8]))  # Extract polygon points
            category = parts[8]  # Extract category
            polygon = Polygon([(coords[i], coords[i+1]) for i in range(0, 8, 2)])
            objects.append((polygon, category))
    return objects

def compute_iou(poly1, poly2):
    """Compute Intersection over Union (IoU) between two polygons."""
    intersection = poly1.intersection(poly2).area
    union = poly1.union(poly2).area
    return intersection / union if union > 0 else 0

def calculate_miou(folder1, folder2):
    """Calculate mean IoU between two annotation folders."""
    files1 = {os.path.basename(f): f for f in glob.glob(os.path.join(folder1, '*.txt'))}
    files2 = {os.path.basename(f): f for f in glob.glob(os.path.join(folder2, '*.txt'))}

    common_files = set(files1.keys()) & set(files2.keys())
    total_iou = []
    
    for file in common_files:
        objects1 = parse_dota_annotation(files1[file])
        objects2 = parse_dota_annotation(files2[file])
        
        matched_ious = []
        
        for poly1, category1 in objects1:
            best_iou = 0
            for poly2, category2 in objects2:
                if category1 == category2:  # Match by category
                    iou = compute_iou(poly1, poly2)
                    best_iou = max(best_iou, iou)
            matched_ious.append(best_iou)
        
        total_iou.extend(matched_ious)
    
    return np.mean(total_iou) if total_iou else 0

# Example Usage
miou = calculate_miou(ann_dir, pred_dir)
print(f"Mean IoU between {ann_dir} and {pred_dir}: {miou:.4f}")


Mean IoU between ./DOTAv10/data_train_2_5_gb/split_ss_dota_1024_200/val/annfiles and ./DOTAv10/data_train_2_5_gb/split_ss_dota_1024_200/val/pseudo_obb_labelTxt_dota_pointobb: 0.3124


### Using linear sum assignment from scipy

In [ ]:
import os
import glob
import numpy as np
from shapely.geometry import Polygon
from scipy.optimize import linear_sum_assignment

def parse_dota_annotation(file_path):
    """Parse a DOTA annotation file and return a list of polygons with categories."""
    objects = []
    with open(file_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) < 9:
                continue
            coords = list(map(float, parts[:8]))  # Extract polygon points
            category = parts[8]  # Extract category
            polygon = Polygon([(coords[i], coords[i+1]) for i in range(0, 8, 2)])
            objects.append((polygon, category))
    return objects

def compute_iou(poly1, poly2):
    """Compute Intersection over Union (IoU) between two polygons."""
    intersection = poly1.intersection(poly2).area
    union = poly1.union(poly2).area
    return intersection / union if union > 0 else 0

def match_objects(objects1, objects2):
    """Perform one-to-one matching using the Hungarian algorithm."""
    if not objects1 or not objects2:
        return []

    iou_matrix = np.zeros((len(objects1), len(objects2)))

    for i, (poly1, cat1) in enumerate(objects1):
        for j, (poly2, cat2) in enumerate(objects2):
            if cat1 == cat2:  # Match by category
                iou_matrix[i, j] = compute_iou(poly1, poly2)
    
    # Convert IoU to a cost matrix (negative IoU for maximization)
    cost_matrix = -iou_matrix  
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    matched_ious = [iou_matrix[i, j] for i, j in zip(row_ind, col_ind) if iou_matrix[i, j] > 0]
    
    return matched_ious

def calculate_miou(folder1, folder2):
    """Calculate mean IoU between two annotation folders using optimal matching."""
    files1 = {os.path.basename(f): f for f in glob.glob(os.path.join(folder1, '*.txt'))}
    files2 = {os.path.basename(f): f for f in glob.glob(os.path.join(folder2, '*.txt'))}

    common_files = set(files1.keys()) & set(files2.keys())
    total_iou = []
    
    for file in common_files:
        objects1 = parse_dota_annotation(files1[file])
        objects2 = parse_dota_annotation(files2[file])
        
        matched_ious = match_objects(objects1, objects2)
        total_iou.extend(matched_ious)
    
    return np.mean(total_iou) if total_iou else 0

miou = calculate_miou(ann_dir, pred_dir)
print(f"Mean IoU between {ann_dir} and {pred_dir}: {miou:.4f}")

Mean IoU between ./DOTAv10/data_train_2_5_gb/split_ss_dota_1024_200/val/annfiles and ./DOTAv10/data_train_2_5_gb/split_ss_dota_1024_200/val/pseudo_obb_labelTxt_dota_pointobb: 0.3096
